<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/Cinema_Seats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col,concat_ws,collect_set,row_number,count

# Initialize Spark session
spark = SparkSession.builder.appName("CinemaSeats").getOrCreate()

# Data for the DataFrame
data = [
    (1, 1), (2, 0), (3, 1), (4, 1), (5, 1),
    (6, 0), (7, 1), (8, 1), (9, 0), (10, 1),
    (11, 0), (12, 1), (13, 0), (14, 1), (15, 1),
    (16, 0), (17, 1), (18, 1), (19, 1), (20, 1)
]

# Define schema
columns = ["seat_id", "free"]

# Create DataFrame
cinema_df = spark.createDataFrame(data, columns)

# Show the DataFrame
#cinema_df.show()

window_spec = Window.partitionBy("free").orderBy("seat_id");

cinema_df2 = cinema_df.filter(col("free") ==1).withColumn("grp",col("seat_id") - row_number().over(window_spec))

cinema_df2.show()
cinema_df3 = cinema_df2.groupBy("grp").agg(count("seat_id").alias("morethanone")).filter(col("morethanone")>1)

cinema_df2 = cinema_df2.join(cinema_df3,on="grp",how="inner").select(cinema_df2["seat_id"],cinema_df3["grp"])

#Cd = cinema_df2.groupBy("grp").agg(count("seat_id").alias("seats"))
cinema_df3 = cinema_df2.groupBy("grp").agg(concat_ws(",",collect_set("seat_id")).alias("seats"))

cinema_df3.show()









+-------+----+---+
|seat_id|free|grp|
+-------+----+---+
|      1|   1|  0|
|      3|   1|  1|
|      4|   1|  1|
|      5|   1|  1|
|      7|   1|  2|
|      8|   1|  2|
|     10|   1|  3|
|     12|   1|  4|
|     14|   1|  5|
|     15|   1|  5|
|     17|   1|  6|
|     18|   1|  6|
|     19|   1|  6|
|     20|   1|  6|
+-------+----+---+

+---+-----------+
|grp|      seats|
+---+-----------+
|  6|19,20,17,18|
|  5|      15,14|
|  1|      5,3,4|
|  2|        7,8|
+---+-----------+

